* Let $\phi(i, j) = i\cdot n + j$ for $i, j \in \{0, \ldots, n-1\}$.
Let $\psi(i, j)=i \cdot n^2 + j$ for $i, j \in \{0, \ldots, n^2-1\}$.
* Let $vec(A)$ for a matrix of size $n\times n$ be a vector of length $n^2$ such that
$vec(A)_{i \cdot n + j} = A_{ij}$ (assuming we start the indexing from zero).
* Let $L$ and $R$ be matrices of size $m \times n^2$.
* Let $A_i$ be a horizontal vector of $i$-th row of the matrix $A$.
* Let $M$ be a matrix of size $m \times n^4$, such that
$M_{a, \psi(b, c)} = L_{a, b} \cdot R_{a, c}$ for every $a, b, c$, i.e. $M_a = vec(L_a^T \cdot R_a)$.
* Let $Q$ be a matrix of size $n^2 \times m$.
* Let $D$ be a matrix of size $n^2 \times n^4$ such that $D_{a} = Q_{a} \cdot M$, i.e. $D = Q \cdot M$.
* Let $C$ be a matrix of size $n^2 \times n^4$ such that
$C_{\phi(i,k), \psi(\phi(i,j), \phi(j,k))} = 1$ and zero otherwise.

In [48]:
from pprint import pprint

from mip import *

def f(i, j, n):
    return i * n + j

def finv(x, n):
    return divmod(x, n)

def find_coeffients(n, m):
    def phi(i, j):
        return f(i, j, n)

    def psi(i, j):
        return f(i, j, n**2)

    model = Model(sense=MINIMIZE)
    L = [[model.add_var(var_type=INTEGER, lb=-1, ub=1) for j in range(n**2)] for i in range(m)]
    R = [[model.add_var(var_type=INTEGER, lb=-1, ub=1) for j in range(n**2)] for i in range(m)]
    M = [[model.add_var(lb=-INF) for j in range(n**4)] for i in range(m)]

    for a in range(m):
        for i in range(n**2):
            for j in range(n**2):
                pass
                # model += M[a][psi(i, j)] == L[a][i] * R[a][j]

    status = model.optimize(max_seconds=300)
    if status == OptimizationStatus.OPTIMAL:
        print('optimal solution cost {} found'.format(model.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        print('sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('no feasible solution found, lower bound is: {}'.format(model.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        print('solution:')
        for v in model.vars:
           print('{} : {}'.format(v.name, v.x))

#  result = find_coeffients(2, 7)

In [65]:
def xor(a, b):
    model = Model()
    x = model.add_var("x", var_type=INTEGER, lb=0, ub=1)
    y = model.add_var("y", var_type=INTEGER, lb=0, ub=1)
    z = model.add_var("z", var_type=INTEGER, lb=0, ub=1)
    p1 = model.add_var("p1", var_type=INTEGER, lb=0, ub=1)
    p2 = model.add_var("p2", var_type=INTEGER, lb=0, ub=1)
    blank = model.add_var("blank", var_type=INTEGER)
    model += x == a
    model += y == b
    # not x :: z = 1 - x
    # x or y :: z >= x, z >= y, z <= x + y
    # x and y :: z <= x, z <= y, x + y <= 1 + z
    # x xor y = (x and not y) or (not x and y)

    model += z >= p1
    model += z >= p2
    model += z <= p1 + p2
    model += p1 <= x
    model += p1 <= 1 - y
    model += x + 1 - y <= 1 + p1
    model += p2 <= 1 - x
    model += p2 <= y
    model += 1 - x + y <= 1 + p2

    model.objective = minimize(blank)

    model.x_gap = 0.05
    status = model.optimize(max_seconds=300)
    # if status == OptimizationStatus.OPTIMAL:
    #     print('optimal solution cost {} found'.format(model.objective_value))
    # elif status == OptimizationStatus.FEASIBLE:
    #     print('sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
    # elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    #     print('no feasible solution found, lower bound is: {}'.format(model.objective_bound))
    # if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    #     print('solution:')
    #     for v in model.vars:
    #        print('{} : {}'.format(v.name, v.x))
    return z.x

for a in range(2):
    for b in range(2):
        print(f"xor({a}, {b}) = {xor(a, b)}")

xor(0, 0) = 0.0
xor(0, 1) = 1.0
xor(1, 0) = 1.0
xor(1, 1) = 0.0


In [69]:
def product(a, b):
    model = Model()
    x = model.add_var("x", var_type=INTEGER, lb=-1, ub=1)
    y = model.add_var("y", var_type=INTEGER, lb=-1, ub=1)
    z = model.add_var("z", var_type=INTEGER, lb=-1, ub=1)

    model += x == a
    model += y == b

    # z == 1 <-> x == 1 and y = 1 or x == -1 and y == -1
    # x == y <-> z == 1 ? not xor
    # p >= x - y, p >= y - x

    status = model.optimize(max_seconds=300)
    if status == OptimizationStatus.OPTIMAL:
        print('optimal solution cost {} found'.format(model.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        print('sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('no feasible solution found, lower bound is: {}'.format(model.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        print('solution:')
        for v in model.vars:
           print('{} : {}'.format(v.name, v.x))

    return z.x

for a in range(-1, 2):
    for b in range(-1, 2):
        print(f"product({a}, {b}) = {product(a, b)}, expected: {a * b}\t{'OK' if product(a, b) == a * b else 'FAIL' }")

optimal solution cost 0.0 found
solution:
x : -1.0
y : -1.0
z : -1.0
optimal solution cost 0.0 found
solution:
x : -1.0
y : -1.0
z : -1.0
product(-1, -1) = -1.0, expected: 1	FAIL
optimal solution cost 0.0 found
solution:
x : -1.0
y : 0.0
z : -1.0
optimal solution cost 0.0 found
solution:
x : -1.0
y : 0.0
z : -1.0
product(-1, 0) = -1.0, expected: 0	FAIL
optimal solution cost 0.0 found
solution:
x : -1.0
y : 1.0
z : -1.0
optimal solution cost 0.0 found
solution:
x : -1.0
y : 1.0
z : -1.0
product(-1, 1) = -1.0, expected: -1	OK
optimal solution cost 0.0 found
solution:
x : 0.0
y : -1.0
z : -1.0
optimal solution cost 0.0 found
solution:
x : 0.0
y : -1.0
z : -1.0
product(0, -1) = -1.0, expected: 0	FAIL
optimal solution cost 0.0 found
solution:
x : 0.0
y : 0.0
z : -1.0
optimal solution cost 0.0 found
solution:
x : 0.0
y : 0.0
z : -1.0
product(0, 0) = -1.0, expected: 0	FAIL
optimal solution cost 0.0 found
solution:
x : 0.0
y : 1.0
z : -1.0
optimal solution cost 0.0 found
solution:
x : 0.0
y : 